# Sorting Algorithms

Sorting is a fundamental operation in many computing system and there are myriad sorting algorithms that vary both in how they work and how they perform.

Let's use use CFiddle to compare several sorting algorthims, implemented in [sorting/sort.cpp](sorting/sort.cpp).

To start, we'll compile `sort.cpp`:

In [ ]:
%xmode Minimal
from cfiddle import *
import pandas as pd

sort = build("sorting/sort.cpp", build_parameters=arg_map(OPTIMIZE="-O3"), verbose=True)[0]

## Bubble Sort

Then we can get started with [bubble sort](https://en.wikipedia.org/wiki/Bubble_sort):

In [ ]:
sort.source("bubble_sort")

Bubble sort's complexity is $O(n^2)$, which we can verify by [plotting](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.line.html) it's execution time as a function of $n$:

In [ ]:
r = run(executable=sort, function="bubble_sort", arguments=arg_map(size=exp_range(4,2**16, 2))).as_df()
r.plot.line(x="size", y="ET", xlabel="n", ylabel="exec. time (s)")

## Insertion Sort

[Insertion Sort](https://en.wikipedia.org/wiki/Insertion_sort) is also $O(n^2)$.  Let's compare them!

In [ ]:
sort.source("insertion_sort")

In [ ]:
r = run(executable=sort, function=["bubble_sort", "insertion_sort"], arguments=arg_map(size=exp_range(4,2**16, 2))).as_df()


This gives a nice table of values:

In [ ]:
display(r)

But it's not so easy to graph.  Panda's [pivot_table](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html) can fix that:

In [ ]:
comparison = pd.pivot_table(r, values="ET", columns="function", index="size")
display(comparison)

In [ ]:
comparison.plot.line(xlabel="n", ylabel="exec. time (s)")

I guess that's why you're not supposed to use bubble sort!

## More Sorts

Let's drop bubble sort because it's terribly slow.

In [ ]:
r = run(executable=sort, function=["insertion_sort", 
                         "merge_sort", 
                         "stl_sort", 
                         "quick_sort"
                        ], arguments=arg_map(size=exp_range(4,2**16, 2))).as_df()

In [ ]:
comparison = pd.pivot_table(r, values="ET", columns="function", index="size")
comparison.plot.line(xlabel="n", ylabel="exec. time (s)")

That really drives home the difference between $O(n^2)$ and $O(n \lg n)$.  Here's just the $O(n \lg n)$ algorthims:

That's a little noisy.  Let's look at larger data sizes:

In [ ]:
r = run(executable=sort, function=[
                         "merge_sort", 
                         "stl_sort", 
                         "quick_sort"], arguments=arg_map(size=exp_range(16,2**16,2))).as_df()
comparison = pd.pivot_table(r, values="ET", columns="function", index="size")
comparison.plot.line(xlabel="n", ylabel="exec. time (s)")

If the numbers look odd, try re-running it.  Performance variation happens!

# The Impact of The Compiler

We can also examine the impact of the compiler on performance.  Let's see how `std::sort()` performance changes with different optimizations.

In [ ]:
sorts = build("sorting/sort.cpp", build_parameters=arg_map(OPTIMIZE=["-O0", "-O1 -fno-inline", "-O1", "-O3"]))
r = run(executable=sorts, function="stl_sort", arguments=arg_map(size=[2**20])).as_df()

In [ ]:
display(r)
comparison = pd.pivot_table(r, values="ET", columns="OPTIMIZE", index="size")
r.plot.bar(y="ET",x="OPTIMIZE", ylabel="exec. time (s)")